In [1]:
'''
Подключение всех нужных библиотек и функций
'''


import numpy as np
import pandas as pd
from scipy.stats import norm
from sklearn.model_selection import train_test_split
import math
from IPython.display import display

In [2]:
np.random.seed(42)  # Начальное условие для генератора случайных чисел


'''
Генерация случайной выборки из многомерного нормального распределения
'''
def gen_N(mean, Sigma, size):
    return np.random.multivariate_normal(mean=mean, cov=Sigma, size=size)


'''
Анализ обучающих и тестовых выборок
'''
def analyze(X1, X2, test_part_1, test_part_2, title):
    # Выборочные средние
    def print_mu_hat():
        print('Выборочные средние:')
        print('mu_hat1 = ' + str(mu_hat1))
        print('mu_hat2 = ' + str(mu_hat2))
        print()
        
    # Вычисление матрицы ковариаций
    def cov_matrix(X):
        return np.cov(X.T)
    
    # Вычисление оценки выборочного среднего
    def mu_hat(X):
        return np.mean(X, axis=0)
    
    # Выборочная S
    def print_S():
        print('Выборочная матрица ковариаций S:')
        print(S)
        print()
    
    # Оценка вектора alpha
    def print_a():
        print('Оценка вектора параметров ДФ:')
        print('a = ' + str(a))
        print()
    
    # Вычисление вектора оценок значений ДФ (значение оценки для каждого вектора из выборки X)
    def DF_values(a, X):
        n = X.shape[0]
        z = np.empty((n, 1))

        for i in range(n):
            z[i] = np.dot(a, X[i])

        return z
    
    # Оценка выборочной несмещённой дисперсии
    def get_s_z_sqr(a, S):
        p = S.shape[0]
        s = 0

        for l in range(p):
            for j in range(p):
                s += a[l] * S[l][j] * a[j]

        return s
        
    # Выборочные средние значения ДФ
    def print_z_mean():
        print('Выборочные средние значения ДФ (векторы z):')
        print('z1_mean = ' + str(z1_mean))
        print('z2_mean = ' + str(z2_mean))
        print()

    # Выборочная несмещённая дисперсия
    def print_s_z_sqr():
        print('Выборочная несмещённая дисперсия:')
        print('s_z_sqr = ' + str(s_z_sqr))
        print()
       
    # Смещённая оценка расстояния Махаланобиса
    def Mahalanobis_distance(z1_mean, z2_mean, s_z_sqr):
        return ((z1_mean - z2_mean)**2) / s_z_sqr

    # Несмещённая оценка расстояния Махаланобиса
    def displaced_Mahalanobis_distance(n1, n2, D_sqr, p):
        return ((n1 + n2 - p - 3) / (n1 + n2 - 2)) * D_sqr - p * (1 / n1 + 1 / n2)
        
    # Оценки расстояния Махаланобиса
    def print_D():
        print('Смещённая оценка расстояния Махаланобиса:')
        print('D_sqr = ' + str(D_sqr))
        print()

        print('Несмещённая оценка расстояния Махаланобиса')
        print('D_H_sqr = ' + str(D_H_sqr))
        print()
        
    # Функция Лапласа Ф(x)
    def F(x):
        return norm.cdf(x)
    
    # Матрица соответствий
    def display_correspondence_table(z1, z2):
        table = [[z1[z1 >= c].shape[0], z1[z1 < c].shape[0]],
                 [z2[z2 >= c].shape[0], z2[z2 < c].shape[0]]]

        correspondence_table = pd.DataFrame([], columns=['1', '2'])
        correspondence_table.columns.name = 'True↓ \ Pred→'
        correspondence_table.loc['1'] = table[0]
        correspondence_table.loc['2'] = table[1]

        display(correspondence_table)

        return table

    # Табличные P
    def print_P_table(table):
        P_2_1_table = table[0][1] / (table[0][0] + table[0][1])
        P_1_2_table = table[1][0] / (table[1][0] + table[1][1])

        print('Табличные оценки вероятности ошибочной классификации:')
        print('P(2|1) = ' + str(P_2_1_table))
        print('P(1|2) = ' + str(P_1_2_table))
        print()

    # Теоретические P
    def print_P_hat():
        P_2_1_hat = F((K - D_H_sqr / 2) / math.sqrt(D_H_sqr))
        P_1_2_hat = F((-K - D_H_sqr / 2) / math.sqrt(D_H_sqr))

        print('Теоретические оценки вероятности ошибочной классификации:')
        print('P^(2|1) = ' + str(P_2_1_hat))
        print('P^(1|2) = ' + str(P_1_2_hat))
        print()

    print(title + '\n\n\nОбучающая выборка\n')
    
    n1, n2, p = X1.shape[0], X2.shape[0], X1.shape[1]
    q1 = n1 / (n1 + n2)
    q2 = n2 / (n1 + n2)

    mu_hat1 = mu_hat(X1)
    mu_hat2 = mu_hat(X2)
    print_mu_hat()
    
    S1 = cov_matrix(X1)
    S2 = cov_matrix(X2)
    S = ((n1 - 1) * S1 + (n2 - 1) * S2) / (n1 + n2 - 2)
    print_S()

    a = np.linalg.solve(S, mu_hat1 - mu_hat2)
    print_a()
    
    z1 = DF_values(a, X1)
    z2 = DF_values(a, X2)
    test_part_1_z = DF_values(a, test_part_1)
    test_part_2_z = DF_values(a, test_part_2)
    z1_mean = np.mean(z1)
    z2_mean = np.mean(z2)
    print_z_mean()
    
    s_z_sqr = get_s_z_sqr(a, S)
    print_s_z_sqr()

    K = math.log(q2 / q1)
    c = (z1_mean + z2_mean) / 2 + K
    D_sqr = Mahalanobis_distance(z1_mean, z2_mean, s_z_sqr)
    D_H_sqr = displaced_Mahalanobis_distance(n1, n2, D_sqr, p)
    print_D()

    table = display_correspondence_table(z1, z2)
    print_P_table(table)
    print_P_hat()

    print('Тестовая выборка\n')
    
    test_table = display_correspondence_table(test_part_1_z, test_part_2_z)
    print_P_table(test_table)

In [3]:
'''
"Хорошо" разделённые данные
'''


sigma_good = np.array(
    [[0, 1, 2],
     [1, 2, 2],
     [2, 2, 3]]
)
mu1_good = np.array([0, 1, 1])
mu2_good = np.array([8, 10, 10])
n1_good = 700
n2_good = 300
test_part_n_good = 150

X1_good = gen_N(mu1_good, sigma_good, n1_good)
X2_good = gen_N(mu2_good, sigma_good, n2_good)
test_X1_good = gen_N(mu1_good, sigma_good, test_part_n_good)
test_X2_good = gen_N(mu2_good, sigma_good, test_part_n_good)

analyze(X1_good, X2_good, test_X1_good, test_X2_good, '"Хорошо" разделённые данные')

"Хорошо" разделённые данные


Обучающая выборка

Выборочные средние:
mu_hat1 = [-0.05777405  0.86015499  0.84656897]
mu_hat2 = [ 8.02021642  9.99600578 10.08433936]

Выборочная матрица ковариаций S:
[[1.49109604 0.89820765 1.04830122]
 [0.89820765 1.85675611 1.87657338]
 [1.04830122 1.87657338 3.31858028]]

Оценка вектора параметров ДФ:
a = [-3.50826214 -3.57042205  0.34355175]

Выборочные средние значения ДФ (векторы z):
z1_mean = -2.5775895675657754
z2_mean = -60.3624886562958

Выборочная несмещённая дисперсия:
s_z_sqr = 57.78489908873

Смещённая оценка расстояния Махаланобиса:
D_sqr = 57.784899088730064

Несмещённая оценка расстояния Махаланобиса
D_H_sqr = 57.539010572485516



<ipython-input-2-92fd3cd15884>:8: RuntimeWarning: covariance is not positive-semidefinite.
  return np.random.multivariate_normal(mean=mean, cov=Sigma, size=size)


True↓ \ Pred→,1,2
1,700,0
2,0,300


Табличные оценки вероятности ошибочной классификации:
P(2|1) = 0.0
P(1|2) = 0.0

Теоретические оценки вероятности ошибочной классификации:
P^(2|1) = 4.7225015256014906e-05
P^(1|2) = 0.00011614993384681894

Тестовая выборка



True↓ \ Pred→,1,2
1,150,0
2,0,150


Табличные оценки вероятности ошибочной классификации:
P(2|1) = 0.0
P(1|2) = 0.0



In [4]:
'''
"Плохо" разделённые данные
'''


sigma_bad = np.array(
    [[4, 2, 8],
     [2, 4, 5],
     [8, 5, 9]]
)
mu1_bad = np.array([1, 0, -2])
mu2_bad = np.array([2, 0, 3])
n1_bad = 800
n2_bad = 200
test_part_n_bad = 350

X1_bad = gen_N(mu1_bad, sigma_bad, n1_bad)
X2_bad = gen_N(mu2_bad, sigma_bad, n2_bad)
test_X1_bad = gen_N(mu1_bad, sigma_bad, test_part_n_bad)
test_X2_bad = gen_N(mu2_bad, sigma_bad, test_part_n_bad)

analyze(X1_bad, X2_bad, test_X1_bad, test_X2_bad, '"Плохо" разделённые данные')

"Плохо" разделённые данные


Обучающая выборка

Выборочные средние:
mu_hat1 = [ 1.09991293  0.13636207 -1.80682513]
mu_hat2 = [2.1648783  0.35765952 3.2334832 ]

Выборочная матрица ковариаций S:
[[ 6.80322235  3.06046832  6.27993361]
 [ 3.06046832  4.43325411  4.28506575]
 [ 6.27993361  4.28506575 10.96010403]]

Оценка вектора параметров ДФ:
a = [ 0.46845784  0.53148503 -0.93608973]

Выборочные средние значения ДФ (векторы z):
z1_mean = 2.2790876787697436
z2_mean = -1.8225855007447112

Выборочная несмещённая дисперсия:
s_z_sqr = 4.1016731795144565

Смещённая оценка расстояния Махаланобиса:
D_sqr = 4.101673179514453

Несмещённая оценка расстояния Махаланобиса
D_H_sqr = 4.066483607652672



<ipython-input-2-92fd3cd15884>:8: RuntimeWarning: covariance is not positive-semidefinite.
  return np.random.multivariate_normal(mean=mean, cov=Sigma, size=size)


True↓ \ Pred→,1,2
1,763,37
2,73,127


Табличные оценки вероятности ошибочной классификации:
P(2|1) = 0.04625
P(1|2) = 0.365

Теоретические оценки вероятности ошибочной классификации:
P^(2|1) = 0.044968147981365295
P^(1|2) = 0.3741739460516746

Тестовая выборка



True↓ \ Pred→,1,2
1,338,12
2,127,223


Табличные оценки вероятности ошибочной классификации:
P(2|1) = 0.03428571428571429
P(1|2) = 0.3628571428571429



In [5]:
'''
Анализ данных из репозитория
'''


'''
Получение и считывание данных из файла в двумерный список int'ов
'''
def parse_file(filename):
    arr = []

    with open(filename) as file:
        for line in file:
            arr.append(list(map(int, line.strip().split())))

    return arr


'''
Разделение выборки X из репозитория на X1 (вектора из 1-го класса) и X2 (из 2-го)
'''
def split_X(X, arr, offset=0):
    p = X.shape[1]
    X1 = np.empty((0, p))
    X2 = np.empty((0, p))

    for i in range(len(X)):
        if arr[i + offset][-1] == 1:
            X1 = np.append(X1, [X[i]], axis=0)
        else:
            X2 = np.append(X2, [X[i]], axis=0)

    return X1, X2



arr = parse_file('german.data-numeric')

n = 1000
p = 24
n1 = 700
n2 = 300
q1 = n1 / n
q2 = n2 / n
X = np.empty((0, p))

for i in range(n):
    x = np.array(arr[i][:-1])
    X = np.append(X, [x], axis=0)

train_X, test_X = train_test_split(X, shuffle=False)
X1, X2 = split_X(train_X, arr)
test_X1, test_X2 = split_X(test_X, arr, offset=train_X.shape[0])

analyze(X1, X2, test_X1, test_X2, 'Данные из репозитория')

Данные из репозитория


Обучающая выборка

Выборочные средние:
mu_hat1 = [2.84629981e+00 1.89981025e+01 2.68311195e+00 2.94022770e+01
 2.22960152e+00 3.47248577e+00 2.72675522e+00 2.81593928e+00
 2.28462998e+00 3.58709677e+01 2.71537002e+00 1.40037951e+00
 1.14800759e+00 1.40417457e+00 1.04554080e+00 1.97343454e-01
 1.17647059e-01 9.10815939e-01 3.03605313e-02 1.49905123e-01
 7.51423150e-01 1.70777989e-02 2.04933586e-01 6.43263757e-01]
mu_hat2 = [1.94170404e+00 2.45784753e+01 2.14798206e+00 3.81883408e+01
 1.70852018e+00 3.13901345e+00 2.56053812e+00 2.81165919e+00
 2.56502242e+00 3.37130045e+01 2.56502242e+00 1.37219731e+00
 1.15246637e+00 1.37219731e+00 1.00896861e+00 2.91479821e-01
 4.93273543e-02 9.05829596e-01 6.27802691e-02 2.28699552e-01
 6.32286996e-01 2.24215247e-02 1.92825112e-01 5.96412556e-01]

Выборочная матрица ковариаций S:
[[ 1.37004337e+00  1.16800106e-01  1.50074646e-01 -5.47688894e-02
   2.71134339e-01  8.70033718e-02 -1.18259784e-02 -7.66809477e-02
   1.52410694e-02

True↓ \ Pred→,1,2
1,473,54
2,113,110


Табличные оценки вероятности ошибочной классификации:
P(2|1) = 0.10246679316888045
P(1|2) = 0.5067264573991032

Теоретические оценки вероятности ошибочной классификации:
P^(2|1) = 0.09350320052513467
P^(1|2) = 0.5575941912890768

Тестовая выборка



True↓ \ Pred→,1,2
1,157,16
2,35,42


Табличные оценки вероятности ошибочной классификации:
P(2|1) = 0.09248554913294797
P(1|2) = 0.45454545454545453

